# Workshop de Python

Cargamos datasets con información de admisiones a hospitales de enfermos de diabetes. El objetivo es, una vez limpiado el dataset, estudiarlo para extraer el máximo número de insights de los datos.

## Cargar librerías

In [306]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from collections import defaultdict
from sklearn.model_selection import train_test_split

## Lectura de datos
- diabetic_data.csv
- admission_source_id.csv
- admission_type_id.csv
- discharge_disposition_id.csv

In [211]:
admin_src_df = pd.read_csv('data/admission_source_id.csv', 
                           na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
admin_type_df = pd.read_csv('data/admission_type_id.csv',
                           na_values = ["Not Available","None","Not Mapped"])
diabetic_df = pd.read_csv('data/diabetic_data.csv',
                          na_values=["Not Available","None","Unknown/Invalid","Not Mapped","?"])
dschrge_disp_df = pd.read_csv('data/discharge_disposition_id.csv',
                             na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
#id_mapping_df = pd.read_csv('data/IDs_mapping.csv')    #Es la colección de todas las demás tabals de referencia

C:\Users\bdih7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [212]:
diabetic_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [213]:
#pandas_profiling.ProfileReport(diabetic_df)

### comprobar correcta lectura de los datos
Revisemos qué hay en cada uno de los dataframes

In [214]:
#admin_src_df
#admin_type_df
dschrge_disp_df
#diabetic_df.head()

,discharge_disposition_id,description
0,1,Discharged to home
1,2,Discharged/transferred to another short term h...
2,3,Discharged/transferred to SNF
3,4,Discharged/transferred to ICF
4,5,Discharged/transferred to another type of inpa...
5,6,Discharged/transferred to home with home healt...
6,7,Left AMA
7,8,Discharged/transferred to home under care of H...
8,9,Admitted as an inpatient to this hospital
9,10,Neonate discharged to another hospital for neo...


Asignemos los índices originales a los ficheros de las tabals de referencia para desnormalizar la tabla _diabetic_ _ _df_

In [215]:
admin_src_df.set_index("admission_source_id", inplace = True)
admin_type_df.set_index("admission_type_id", inplace = True)
dschrge_disp_df.set_index("discharge_disposition_id", inplace = True)

Cambiemos el nombre de las descripciones de las tablas de referencia

In [216]:
admin_src_df.rename(columns={"description":"admin_src_desc"}, inplace = True)
admin_type_df.rename(columns={"description":"admin_type_desc"}, inplace = True)
dschrge_disp_df.rename(columns={"description":"dschrge_disp_desc"}, inplace = True)

### juntar todos los datos en el mismo DataFrame

Ahora sí ejecutemos los joins. Verifiquemos que si las relaciones son en efecto uno a uno

In [217]:
print(diabetic_df.join(admin_src_df,how="left", on = "admission_source_id").shape)
print(diabetic_df.join(admin_type_df,how="left", on = "admission_type_id").shape)
print(diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id").shape)
print(diabetic_df.shape)

(101766, 51)
(101766, 51)
(101766, 51)
(101766, 50)


In [218]:
aug_diabetic_df = diabetic_df.join(admin_src_df,how="left", on="admission_source_id")
aug_diabetic_df = aug_diabetic_df.join(admin_type_df,how="left", on = "admission_type_id")
aug_diabetic_df = aug_diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id")

Quitemos los id's que ya denormalizamos

In [219]:
aug_diabetic_df.drop(labels = ["admission_type_id","discharge_disposition_id","admission_source_id"], 
                     axis = 1, 
                     inplace = True)

In [220]:
aug_diabetic_df

,encounter_id,patient_nbr,race,gender,age,weight,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admin_src_desc,admin_type_desc,dschrge_disp_desc
0,2278392,8222157,Caucasian,Female,[0-10),NaN,1,NaN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,NO,Physician Referral,NaN,NaN
1,149190,55629189,Caucasian,Female,[10-20),NaN,3,NaN,NaN,59,...,No,No,No,No,Ch,Yes,>30,Emergency Room,Emergency,Discharged to home
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,2,NaN,NaN,11,...,No,No,No,No,No,Yes,NO,Emergency Room,Emergency,Discharged to home
3,500364,82442376,Caucasian,Male,[30-40),NaN,2,NaN,NaN,44,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,NaN,NaN,51,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
5,35754,82637451,Caucasian,Male,[50-60),NaN,3,NaN,NaN,31,...,No,No,No,No,No,Yes,>30,Clinic Referral,Urgent,Discharged to home
6,55842,84259809,Caucasian,Male,[60-70),NaN,4,NaN,NaN,70,...,No,No,No,No,Ch,Yes,NO,Clinic Referral,Elective,Discharged to home
7,63768,114882984,Caucasian,Male,[70-80),NaN,5,NaN,NaN,73,...,No,No,No,No,No,Yes,>30,Emergency Room,Emergency,Discharged to home
8,12522,48330783,Caucasian,Female,[80-90),NaN,13,NaN,NaN,68,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Urgent,Discharged to home
9,15738,63555939,Caucasian,Female,[90-100),NaN,12,NaN,InternalMedicine,33,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Elective,Discharged/transferred to SNF


Ahora entendamos la información que tenemos en el _dataset_

In [221]:
aug_diabetic_df.set_index(["patient_nbr","encounter_id"], inplace = True)

## Data Wrangling

### Cambiar los nombres de las columnas para que cumplan buenas prácticas


In [222]:
aug_diabetic_df.columns = aug_diabetic_df.columns.str.lower().str.strip(' ').str.replace(pat="-",repl="_")

In [223]:
aug_diabetic_df.rename(columns={"diabetesmed":"diabetes_med","a1cresult":"a1c_result"},inplace=True)

### Estudiar el formato de las variables, ver cuáles se deberían modificar y modificarlas cuando se considere oportuno

Basados en el value_count(), convertiremos las siguientes variables a categorías ordenadas. No lo aplicamos a todos los valores enteros ya que algunos de ellos tienen muchos valores

In [224]:
#ordered_categories = ["num_procedures","time_in_hospital","number_inpatient","number_diagnoses"]
#aug_diabetic_df[ordered_categories] = aug_diabetic_df[ordered_categories].apply(lambda x: pd.Categorical(x.values, ordered = True))

Convirtamos ahora las variables codificadas como objetos restantes a categóricas 

In [225]:
aug_diabetic_df[aug_diabetic_df.select_dtypes("object").columns] = aug_diabetic_df[aug_diabetic_df.select_dtypes("object").columns].apply(lambda x: x.astype("category"))#pd.Categorical(x.values,categories = list(x.dropna().unique()) + ["Faltante"]))

### Tratamiento de Outliers

In [226]:
## Número de visitas ambulatorias en el último año. Definimos rangos
number_outpat = []
for i in aug_diabetic_df.number_outpatient:
    if (i == 0):
        number_outpat.append("0")
    elif (i <= 2):
        number_outpat.append("1_2")
    elif (i <= 4):
        number_outpat.append("3_4")
    else:
        number_outpat.append("5_more")

aug_diabetic_df.number_outpatient = number_outpat
del(number_outpat)
aug_diabetic_df.number_outpatient.value_counts()

0         85027
1_2       12141
3_4        3141
5_more     1457
Name: number_outpatient, dtype: int64

In [227]:
## Visitas de emergencia en el último año (anterior al encuentro)
number_emergency = []
for i in aug_diabetic_df.number_emergency:
    if (i == 0):
        number_emergency.append("0")
    elif (i <= 2):
        number_emergency.append("1_2")
    else:
        number_emergency.append("3_more")

aug_diabetic_df.number_emergency = number_emergency
del(number_emergency)
aug_diabetic_df.number_emergency.value_counts()


0         90383
1_2        9719
3_more     1664
Name: number_emergency, dtype: int64

In [228]:
# Número de visitas 
aug_diabetic_df.number_inpatient.value_counts()

number_inpat = []
for i in aug_diabetic_df.number_inpatient:
    if (i == 0):
        number_inpat.append("0")
    elif (i <= 1):
        number_inpat.append("1")
    else:
        number_inpat.append("2_more")

aug_diabetic_df.number_inpatient = number_inpat
del(number_inpat)
aug_diabetic_df.number_inpatient.value_counts()

0         67630
1         19521
2_more    14615
Name: number_inpatient, dtype: int64

### Verifiquemos valores nulos de variables importantes

Antes de revisar valores nulos mal codificados, verifiquemos el total de valores nulos ya identificados

In [229]:
aug_diabetic_df.isna().sum()/aug_diabetic_df.shape[0]

race                        0.022336
gender                      0.000029
age                         0.000000
weight                      0.968585
time_in_hospital            0.000000
payer_code                  0.395574
medical_specialty           0.490822
num_lab_procedures          0.000000
num_procedures              0.000000
num_medications             0.000000
number_outpatient           0.000000
number_emergency            0.000000
number_inpatient            0.000000
diag_1                      0.000206
diag_2                      0.003518
diag_3                      0.013983
number_diagnoses            0.000000
max_glu_serum               0.947468
a1c_result                  0.832773
metformin                   0.000000
repaglinide                 0.000000
nateglinide                 0.000000
chlorpropamide              0.000000
glimepiride                 0.000000
acetohexamide               0.000000
glipizide                   0.000000
glyburide                   0.000000
t

Hasta el momento tenemos algunos _features_ con hasta 94 y 96% de valores nulos. En el caso del nivel de glucosa podríamos eliminarla, pero como es un _feature_ importante para este tipo de problema, el nivel máximo de glucosa, codifiquemos explícitamente el NA en las variables categóricas. En cuánto al peso, este falta por procedimientos administrativos del hospital, ya que muchos no estaba obligados a tenerlas registradas. Por esto eliminemos primero el peso

In [230]:
aug_diabetic_df.drop(["weight"], axis = 1, inplace = True)

Ahora escojamos las variables a las que les cambiaremos sus NA por la categoría "faltante"

In [231]:
na_cat_cols = aug_diabetic_df.select_dtypes(exclude=np.number).columns[aug_diabetic_df.select_dtypes(exclude=np.number).isna().sum() > 0]

In [232]:
aug_diabetic_df_imp = aug_diabetic_df.copy()
aug_diabetic_df_imp[na_cat_cols] = aug_diabetic_df_imp[na_cat_cols].apply(lambda x: x.cat.set_categories(list(x.dropna().unique()) + ["Faltante"]))
aug_diabetic_df_imp[na_cat_cols] = aug_diabetic_df_imp[na_cat_cols].fillna("Faltante")

Dado que las variables numéricas no tienen outliers, por el momento podemos considerar que hemos terminado con nuestra imputación

In [233]:
aug_diabetic_df_imp.isna().sum()/aug_diabetic_df.shape[0]

race                        0.0
gender                      0.0
age                         0.0
time_in_hospital            0.0
payer_code                  0.0
medical_specialty           0.0
num_lab_procedures          0.0
num_procedures              0.0
num_medications             0.0
number_outpatient           0.0
number_emergency            0.0
number_inpatient            0.0
diag_1                      0.0
diag_2                      0.0
diag_3                      0.0
number_diagnoses            0.0
max_glu_serum               0.0
a1c_result                  0.0
metformin                   0.0
repaglinide                 0.0
nateglinide                 0.0
chlorpropamide              0.0
glimepiride                 0.0
acetohexamide               0.0
glipizide                   0.0
glyburide                   0.0
tolbutamide                 0.0
pioglitazone                0.0
rosiglitazone               0.0
acarbose                    0.0
miglitol                    0.0
troglita

### Quitemos las variables con baja varianza

In [234]:
pandas_profiling.ProfileReport(aug_diabetic_df_imp)

Number of variables,49
Number of observations,101766
Total Missing (%),0.0%
Total size in memory,11.9 MiB
Average record size in memory,123.1 B
Numeric,7
Categorical,40
Boolean,0
Date,0
Text (Unique),0
Rejected,2


Las siguientes variables tienen una varianza muy muy baja:
   
   * acetohexamide
   * citoglipton
   * examide
   * glimepiride_pioglitazone
   * metformin_pioglitazone
   * metformin_rosiglitazone

   
Quitemos estas variables

In [235]:
low_var_rm = ["acetohexamide","citoglipton","examide","glimepiride_pioglitazone","metformin_pioglitazone","metformin_rosiglitazone","troglitazone"]
aug_diabetic_df_imp.drop(low_var_rm, axis = 1, inplace = True)

Variables con muy alta variabilidad son:

* diag_1
* diag_2
* diag_3

Podría reducirse el número de categorías (>700) con el conocimiento de un experto

Variables numéricas candidatas a ser categóricas binarias:
* number_emergency
* number_outpatient
 

### Encoding para realizar la agregación

In [237]:
oh_diab_df = aug_diabetic_df_imp.copy()

In [239]:
#oh_encoder = OneHotEncoder(handle_unknown="ignore")
l_bin_dict = defaultdict(LabelBinarizer)
oh_diab_df.select_dtypes(exclude=np.number).drop("readmitted",axis=1).apply(lambda x: l_bin_dict[x.name].fit(x))

race                   LabelBinarizer(neg_label=0, pos_label=1, spars...
gender                 LabelBinarizer(neg_label=0, pos_label=1, spars...
age                    LabelBinarizer(neg_label=0, pos_label=1, spars...
payer_code             LabelBinarizer(neg_label=0, pos_label=1, spars...
medical_specialty      LabelBinarizer(neg_label=0, pos_label=1, spars...
number_outpatient      LabelBinarizer(neg_label=0, pos_label=1, spars...
number_emergency       LabelBinarizer(neg_label=0, pos_label=1, spars...
number_inpatient       LabelBinarizer(neg_label=0, pos_label=1, spars...
diag_1                 LabelBinarizer(neg_label=0, pos_label=1, spars...
diag_2                 LabelBinarizer(neg_label=0, pos_label=1, spars...
diag_3                 LabelBinarizer(neg_label=0, pos_label=1, spars...
max_glu_serum          LabelBinarizer(neg_label=0, pos_label=1, spars...
a1c_result             LabelBinarizer(neg_label=0, pos_label=1, spars...
metformin              LabelBinarizer(neg_label=0, 

In [240]:
def one_hot_encoder(df, binarizer_dict):
    for feature_name, binarizer in binarizer_dict.items():
        if(binarizer.classes_.shape[0] == 2):
            df[feature_name + "_" + list(binarizer.classes_)[0]] = binarizer.transform(df[feature_name]) 
        
        else:
            tmp_df = pd.DataFrame(binarizer.transform(df[feature_name]), columns=[feature_name + "_" + s for s in list(binarizer.classes_)], index = df.index)
            df = pd.concat([df , tmp_df], axis = 1)
    
    return(df)
    #print([feature_name + "_" + s for s in list(binarizer.classes_)])
    #print(pd.DataFrame(binarizer.transform(oh_diab_df[feature_name]), columns=[feature_name + "_" + s for s in list(binarizer.classes_)]))


In [241]:
one_hot_encoder(oh_diab_df,l_bin_dict)

,,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,dschrge_disp_desc_Discharged/transferred/referred to this institution for outpatient services,dschrge_disp_desc_Expired,"dschrge_disp_desc_Expired at home. Medicaid only, hospice.","dschrge_disp_desc_Expired in a medical facility. Medicaid only, hospice.",dschrge_disp_desc_Faltante,dschrge_disp_desc_Hospice / home,dschrge_disp_desc_Hospice / medical facility,dschrge_disp_desc_Left AMA,dschrge_disp_desc_Neonate discharged to another hospital for neonatal aftercare,dschrge_disp_desc_Still patient or expected to return for outpatient services
patient_nbr,encounter_id,,,,,,,,,,,,,,,,,,,,,
8222157,2278392,Caucasian,Female,[0-10),1,Faltante,Pediatrics-Endocrinology,41,0,1,0,...,0,0,0,0,1,0,0,0,0,0
55629189,149190,Caucasian,Female,[10-20),3,Faltante,Faltante,59,0,18,0,...,0,0,0,0,0,0,0,0,0,0
86047875,64410,AfricanAmerican,Female,[20-30),2,Faltante,Faltante,11,5,13,1_2,...,0,0,0,0,0,0,0,0,0,0
82442376,500364,Caucasian,Male,[30-40),2,Faltante,Faltante,44,1,16,0,...,0,0,0,0,0,0,0,0,0,0
42519267,16680,Caucasian,Male,[40-50),1,Faltante,Faltante,51,0,8,0,...,0,0,0,0,0,0,0,0,0,0
82637451,35754,Caucasian,Male,[50-60),3,Faltante,Faltante,31,6,16,0,...,0,0,0,0,0,0,0,0,0,0
84259809,55842,Caucasian,Male,[60-70),4,Faltante,Faltante,70,1,21,0,...,0,0,0,0,0,0,0,0,0,0
114882984,63768,Caucasian,Male,[70-80),5,Faltante,Faltante,73,0,12,0,...,0,0,0,0,0,0,0,0,0,0
48330783,12522,Caucasian,Female,[80-90),13,Faltante,Faltante,68,2,28,0,...,0,0,0,0,0,0,0,0,0,0


# Pipeline de preprocesamiento

Ya con el preprocesamiento definido, definamos un pipeline para preprocesar los datos

In [301]:
aug_diabetic_df.readmitted.value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [317]:
#Cargar el dataset
def load_df(path):
    df = pd.read_csv(path,
                     na_values=["Not Available","None","Unknown/Invalid","Not Mapped","?"])
    return(df)

#Recodificar la variable objetivo
def recode_target(df):
    recode = []
    for i in df.readmitted:
        if (i == ">30"):
            recode.append("YES")
        elif (i == "<30"):
            recode.append("YES")
        else:
            recode.append("NO")

    df.readmitted = recode
    
    return df

#Unir el dataset con las tablas de referencia
def augment_df(df,admin_src_df,admin_type_df,dschrge_disp_df):
    aug_df = df.copy()
    
    #Ejecutar los joins
    aug_df = aug_df.join(admin_src_df,how="left", on="admission_source_id")
    aug_df = aug_df.join(admin_type_df,how="left", on = "admission_type_id")
    aug_df = aug_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id")
    
    #Fijar los indices
    aug_df.set_index(["patient_nbr","encounter_id"], inplace = True)
    
    #Quitar los id's
    aug_df.drop(labels = ["admission_type_id","discharge_disposition_id","admission_source_id"],
                axis = 1, 
                inplace = True)
        
    return(aug_df)

#Ajustar los nomnbres de las variables de acuerdo al estándar definido
def ajustar_nombre_cols(df):
    df.columns = df.columns.str.lower().str.strip(' ').str.replace(pat="-",repl="_")
    df.rename(columns={"diabetesmed":"diabetes_med","a1cresult":"a1c_result"},inplace=True)
    return(df)

#Typecasting de las variables categoricas
def typecasting_cats(df):
    df[df.select_dtypes("object").columns] = df[df.select_dtypes("object").columns].apply(lambda x: x.astype("category"))
    return(df)

#Tratar los outliers de ciertas columnas
def outlier_treatment_to_cat(df):
    #Variables que se van a recodificar a través de las siguientes listas
    number_outpat = []
    number_emergency = []
    number_inpat = []
    
    #Número de citas que tuvo sin ser hospitalizado o number_outpat
    for i in df.number_outpatient:
        if (i == 0):
            number_outpat.append("0")
        elif (i <= 2):
            number_outpat.append("1_2")
        elif (i <= 4):
            number_outpat.append("3_4")
        else:
            number_outpat.append("5_more")

    df.number_outpatient = number_outpat
    
    #Número de veces que fue al hospital por emergencias
    for i in df.number_emergency:
        if (i == 0):
            number_emergency.append("0")
        elif (i <= 2):
            number_emergency.append("1_2")
        else:
            number_emergency.append("3_more")

    df.number_emergency = number_emergency
    
    #Número de veces que fue hospitalizado por lo menos una noche
    for i in df.number_inpatient:
        if (i == 0):
            number_inpat.append("0")
        elif (i <= 1):
            number_inpat.append("1")
        else:
            number_inpat.append("2_more")

    df.number_inpatient = number_inpat

    return(df)

#Funciones para encontrar outliers y ajustarlos
def fit_outliers(df, num_std = 2.96):
    lim_sup = []
    columnas_outliers = df.select_dtypes(include=['float64', 'int64']).columns
    for element in columnas_outliers:
        std = df[element].std()
        mean = df[element].mean()
        lim_sup.append(int(round(mean + num_std * std)))
    
    return columnas_outliers, lim_sup

#Ajustarlos
def transform_outliers(df, cols, lim_sup):
    j = 0
    df2 = df.copy()
    for element in cols:
        df2[element] = [min(x, lim_sup[j]) for x in df[element]]
        j+=1
    return df2

#Quitar variables con muy baja varianza especificada en el vector na_var_rm
def low_vars_rm(df,low_var_vec):
    df.drop(low_var_vec, axis = 1, inplace = True)
    return(df)

#Quitar variables con un número muy alto de NA's especificado en el vector na_var_rm
def na_vars_rm(df,na_var_vec):
    df.drop(na_var_vec, axis = 1, inplace = True)
    return(df)

#Imputar las variables categorícas de forma explícita con la categoría "Faltante"
def imputar_categoricas(df):
    #Lista de variables categoricas con NA
    na_cat_cols = df.select_dtypes(exclude=np.number).columns[df.select_dtypes(exclude=np.number).isna().sum() > 0]
    #Agregar la categoría faltante a las variables categóricas
    df[na_cat_cols] = df[na_cat_cols].apply(lambda x: x.cat.set_categories(list(x.dropna().unique()) + ["Faltante"]))
    #Imputar los valores faltantes con la categoría "Faltante"
    df[na_cat_cols] = df[na_cat_cols].fillna("Faltante")

    return(df)

#Crear el diccionario de binarizers para hacer el one-hot encoding
def crear_diccionario_binarizer(df):
    l_bin_dict = defaultdict(LabelBinarizer)
    binarizer_dict = df.select_dtypes(exclude=np.number).apply(lambda x: l_bin_dict[x.name].fit(x))
    return(binarizer_dict)

In [318]:
def cargar_y_split(path):
    #cargar el dataset
    df = load_df(path=path)
    
    #Codificar variable objetivo
    df = recode_target(df)
    
    #realizar split train
    x_train, x_test, y_train, y_test = train_test_split(df.drop("readmitted",axis=1),
                                                        df.readmitted, 
                                                        random_state = 123, 
                                                        stratify = df.readmitted, 
                                                        test_size = 0.3)
    
    return x_train, x_test, y_train, y_test
    

def pipeline_train(df,admin_src_df,admin_type_df,dschrge_disp_df,low_var_vec,na_var_vec):

    #Enrriquecer el dataset
    df = augment_df(df,admin_src_df,admin_type_df,dschrge_disp_df)
    
    #Ajustar nombres
    df = ajustar_nombre_cols(df)
    
    #Tratar outliers pasando las variables a categóricas
    df = outlier_treatment_to_cat(df)
    
    #Encontrar features para hacerles capping
    columnas_outliers, lim_sup = fit_outliers(df)
    
    #Ejecutar el capping sobre estos features definidos en columnas_outliers
    df = transform_outliers(df, cols = columnas_outliers, lim_sup=lim_sup)
    
    #Typecasting
    df = typecasting_cats(df)
        
    #Quitar variables de baja varianza
    df = low_vars_rm(df,low_var_vec)
    
    #Quitar variables con un número alto de NA's
    df = na_vars_rm(df,na_var_vec)
    
    #Codificar explícitamente los NA's
    df = imputar_categoricas(df)
    
    #Crear diccionario binarizer
    binarizer_dict = crear_diccionario_binarizer(df)
    
    #Realizar one-hot encoding
    df = one_hot_encoder(df, binarizer_dict)
    
    return df, columnas_outliers, lim_sup, binarizer_dict

In [313]:
low_var_rm = ["acetohexamide","citoglipton","examide","glimepiride_pioglitazone","metformin_pioglitazone","metformin_rosiglitazone","troglitazone"]
na_var_rm = ["weight"]

In [314]:
x_train, x_test, y_train, y_test =  cargar_y_split("data/diabetic_data_ws.csv")

C:\Users\bdih7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [319]:
x_train_pre,columnas_outliers, lim_sup_outliers, binarizer_dict = pipeline_train(x_train,admin_src_df,admin_type_df,dschrge_disp_df,low_var_rm,na_var_rm)

In [323]:
x_train_pre

,,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,dschrge_disp_desc_Discharged/transferred/referred to this institution for outpatient services,dschrge_disp_desc_Expired,"dschrge_disp_desc_Expired at home. Medicaid only, hospice.","dschrge_disp_desc_Expired in a medical facility. Medicaid only, hospice.",dschrge_disp_desc_Faltante,dschrge_disp_desc_Hospice / home,dschrge_disp_desc_Hospice / medical facility,dschrge_disp_desc_Left AMA,dschrge_disp_desc_Neonate discharged to another hospital for neonatal aftercare,dschrge_disp_desc_Still patient or expected to return for outpatient services
patient_nbr,encounter_id,,,,,,,,,,,,,,,,,,,,,
-5507571366342522588,231840588,Caucasian,Female,[40-50),3,DM,Faltante,53,6,16,0,...,0,0,0,0,0,0,0,0,0,0
-2121164979740686064,120204594,Caucasian,Female,[60-70),3,MC,Orthopedics,26,2,35,1_2,...,0,0,0,0,0,0,0,0,0,0
6744321317740308718,79176972,AfricanAmerican,Female,[40-50),4,Faltante,InternalMedicine,67,0,16,0,...,0,0,0,0,0,0,0,0,0,0
8769627295344250645,150725256,Caucasian,Male,[70-80),2,MC,Faltante,57,0,17,0,...,0,0,0,0,0,0,0,0,0,0
-661136200660237769,80165568,Caucasian,Male,[50-60),13,Faltante,Faltante,53,5,22,0,...,0,0,0,0,0,0,0,0,0,0
8569064694212585723,176782398,Caucasian,Male,[30-40),5,BC,Faltante,75,2,25,0,...,0,0,0,0,0,0,0,0,0,0
-5971356033009238106,52491708,Hispanic,Male,[60-70),6,Faltante,InternalMedicine,67,1,8,0,...,0,0,0,0,0,0,0,0,0,0
-2200814024863824966,193027038,AfricanAmerican,Female,[60-70),7,MC,Cardiology,58,1,23,0,...,0,0,0,0,0,0,0,0,0,0
-5045355015718688786,122305074,Caucasian,Female,[50-60),2,UN,Faltante,67,0,6,0,...,0,0,0,0,0,0,0,0,0,0


In [321]:
def pipeline_test(df,admin_src_df,admin_type_df,dschrge_disp_df,low_var_vec,na_var_vec,columnas_outliers, lim_sup_outliers, binarizer_dict ):

    #Enrriquecer el dataset
    df = augment_df(df,admin_src_df,admin_type_df,dschrge_disp_df)
    
    #Ajustar nombres
    df = ajustar_nombre_cols(df)
    
    #Tratar outliers pasando las variables a categóricas
    df = outlier_treatment_to_cat(df)
    
    #Ejecutar el capping sobre estos features definidos en columnas_outliers
    df = transform_outliers(df, cols = columnas_outliers, lim_sup = lim_sup_outliers)
    
    #Typecasting
    df = typecasting_cats(df)
        
    #Quitar variables de baja varianza
    df = low_vars_rm(df,low_var_vec)
    
    #Quitar variables con un número alto de NA's
    df = na_vars_rm(df,na_var_vec)
    
    #Codificar explícitamente los NA's
    df = imputar_categoricas(df)
    
    #Realizar one-hot encoding
    df = one_hot_encoder(df, binarizer_dict)
    
    return df

In [322]:
x_test_pre = pipeline_test(x_test,admin_src_df,admin_type_df,dschrge_disp_df,low_var_rm,na_var_rm,columnas_outliers, lim_sup_outliers, binarizer_dict )

,,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,dschrge_disp_desc_Discharged/transferred/referred to this institution for outpatient services,dschrge_disp_desc_Expired,"dschrge_disp_desc_Expired at home. Medicaid only, hospice.","dschrge_disp_desc_Expired in a medical facility. Medicaid only, hospice.",dschrge_disp_desc_Faltante,dschrge_disp_desc_Hospice / home,dschrge_disp_desc_Hospice / medical facility,dschrge_disp_desc_Left AMA,dschrge_disp_desc_Neonate discharged to another hospital for neonatal aftercare,dschrge_disp_desc_Still patient or expected to return for outpatient services
patient_nbr,encounter_id,,,,,,,,,,,,,,,,,,,,,
2426283894956289951,190794822,Caucasian,Male,[70-80),7,SP,Faltante,75,0,21,0,...,0,0,0,0,0,0,0,0,0,0
-4425485599712343560,159380244,Caucasian,Female,[30-40),7,BC,Family/GeneralPractice,38,1,22,1_2,...,0,0,0,0,0,0,0,0,0,0
3916767946203212719,276157686,Caucasian,Male,[60-70),1,MC,Faltante,56,0,16,0,...,0,0,0,0,0,0,0,0,0,0
-7797850003729047951,173567316,Caucasian,Female,[70-80),3,MC,Orthopedics,28,2,25,0,...,0,0,0,0,0,0,0,0,0,0
-8391622307144419464,10671636,Other,Female,[60-70),8,Faltante,Faltante,71,6,38,0,...,0,0,0,0,0,0,0,0,0,0
-4927500230696105487,403800176,Caucasian,Male,[50-60),3,UN,Family/GeneralPractice,1,2,8,0,...,0,0,0,0,0,0,0,0,0,0
-6695738218985670547,268281504,Caucasian,Female,[40-50),7,HM,Faltante,1,0,10,1_2,...,0,0,0,0,0,0,0,0,0,0
8837513076783013802,97596828,AfricanAmerican,Female,[50-60),12,Faltante,InternalMedicine,42,0,22,0,...,0,0,0,0,0,0,0,0,0,0
-3485379172661530502,70139940,Caucasian,Male,[60-70),1,Faltante,Family/GeneralPractice,45,0,5,0,...,0,0,0,0,0,0,0,0,0,0
